# Obtaining the Data/Data Augmentation

- nfl.csv is adapted from a larger Kaggle Dataset (https://www.kaggle.com/datasets/maxhorowitz/nflplaybyplay2009to2016?select=NFL+Play+by+Play+2009-2018+%28v5%29.csv)
- I created new columns to help identify potential patterns to predict whether a pass was a completion or not

In [44]:
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder


df = pd.read_csv("nfl.csv")

encoder = LabelEncoder()

df = pd.get_dummies(df, columns=['pass_location', "pass_length"], drop_first=True)

df["pass_difficulty"] = df["air_yards"] / (df["pass_length_short"] + 1)
df["time_pressure"] = df["quarter_seconds_remaining"] / (df["down"] + 1)



# Preprocessing

The data preprocessing steps include:

1. Handling missing values.
2. Encoding categorical variables
3. Splitting the data into training and testing sets.
4. Normalizing the input features.
5. Handling class imbalance using SMOTE, since there tended to be more completions than incompletions


In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE



df["passer_player_id"] = encoder.fit_transform(df["passer_player_id"])
df["receiver_player_id"] = encoder.fit_transform(df["receiver_player_id"])
df = df.fillna(0)


corr_with_complete = df.corr()["complete_pass"].sort_values(ascending=False)
print(corr_with_complete)


sns.heatmap(df.corr())

scaler = StandardScaler()

X = df.drop(columns=["complete_pass"], axis=1)
y = df["complete_pass"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = np.array(X_train).astype(np.float32)
y_train = np.array(y_train).astype(np.float32)
X_test = np.array(X_test).astype(np.float32)
y_test = np.array(y_test).astype(np.float32)


values, counts = np.unique(y_test, return_counts=True)
print(dict(zip(values, counts)))

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)


# Model

- Input layer with 22 features.
- Three hidden layers with 256, 128, and 64 neurons respectively, each followed by batch normalization and ReLU activation.
- Output layer with a sigmoid activation function.


## Training

The model is compiled with the Adam optimizer and binary cross-entropy loss. The learning rate is scheduled to reduce on plateau, and the model is trained for 10 epochs with a batch size of 32.

## Evaluation

The model is evaluated on the test set, and the test loss and accuracy are printed.

In [ ]:
import tensorflow as tf
from tensorflow import keras


model = keras.Sequential([
    keras.layers.Input(shape=(22,)),
    keras.layers.Dense(256, activation = "relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(128, activation = "relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(64, activation = "relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1, activation = "sigmoid")
])


lr_scheduler = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)


optimizer = keras.optimizers.Adam(learning_rate=0.0005)

model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size = 32, callbacks=[lr_scheduler])

results = model.evaluate(X_test, y_test)
print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1]}")